<a href="https://colab.research.google.com/github/Mo3nb/Job-Recommendation/blob/main/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import numpy as np
from sklearn.model_selection import train_test_split   

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
combinedjob = pd.read_csv('/content/drive/My Drive/DATA /Combined_Jobs_Final.csv',encoding='utf-8')
experience = pd.read_csv('/content/drive/My Drive/DATA /Experience.csv',encoding='utf-8')
jobview = pd.read_csv('/content/drive/My Drive/DATA /Job_Views.csv',encoding='utf-8')
POI = pd.read_csv('/content/drive/My Drive/DATA /Positions_Of_Interest.csv',encoding='utf-8')

In [7]:
combinedjob.isnull().sum()

Job.ID                    0
Provider                  0
Status                    0
Slug                      0
Title                     0
Position                  0
Company                2271
City                    135
State.Name              171
State.Code              171
Address               84054
Latitude                  0
Longitude                 0
Industry              83823
Job.Description          56
Requirements          84090
Salary                83861
Listing.Start           683
Listing.End             167
Employment.Type          10
Education.Required      267
Created.At                0
Updated.At                0
dtype: int64

In [8]:
combinedjob['Education.Required']=combinedjob['Education.Required'].fillna('Not Specified')
combinedjob.groupby(['Education.Required']).count() 

,Job.ID,Provider,Status,Slug,Title,Position,Company,City,State.Name,State.Code,...,Longitude,Industry,Job.Description,Requirements,Salary,Listing.Start,Listing.End,Employment.Type,Created.At,Updated.At
Education.Required,,,,,,,,,,,,,,,,,,,,,
Associate Degree,3483,3483,3483,3483,3483,3483,3381,3478,3480,3480,...,3483,0,3483,0,0,3458,3483,3483,3483,3483
Bachelor's Degree,2849,2849,2849,2849,2849,2849,2753,2848,2847,2847,...,2849,0,2849,0,0,2839,2849,2849,2849,2849
High School Diploma,14250,14250,14250,14250,14250,14250,13704,14137,14127,14127,...,14250,0,14250,0,0,14126,14250,14250,14250,14250
Master's Degree,1059,1059,1059,1059,1059,1059,1009,1059,1059,1059,...,1059,0,1059,0,0,1059,1059,1059,1059,1059
Not Specified,62449,62449,62449,62449,62449,62449,60972,62433,62406,62406,...,62449,267,62393,0,229,61925,62282,62439,62449,62449


In [9]:
#replacing missing value with companies' headquarters location
combinedjob['Company'] = combinedjob['Company'].replace(['Genesis Health Systems'], 'Genesis Health System')
combinedjob.loc[combinedjob.Company == 'CHI Payment Systems', 'City'] = 'Illinois'
combinedjob.loc[combinedjob.Company == 'Academic Year In America', 'City'] = 'Stamford'
combinedjob.loc[combinedjob.Company == 'CBS Healthcare Services and Staffing ', 'City'] = 'Urbandale'
combinedjob.loc[combinedjob.Company == 'Driveline Retail', 'City'] = 'Coppell'
combinedjob.loc[combinedjob.Company == 'Educational Testing Services', 'City'] = 'New Jersey'
combinedjob.loc[combinedjob.Company == 'Genesis Health System', 'City'] = 'Davennport'
combinedjob.loc[combinedjob.Company == 'Home Instead Senior Care', 'City'] = 'Nebraska'
combinedjob.loc[combinedjob.Company == 'St. Francis Hospital', 'City'] = 'New York'
combinedjob.loc[combinedjob.Company == 'Volvo Group', 'City'] = 'Washington'
combinedjob.loc[combinedjob.Company == 'CBS Healthcare Services and Staffing', 'City'] = 'Urbandale'
combinedjob['Job.Description']=combinedjob['Job.Description'].fillna('not mentioned')


##TF TD-IDF based on job description ==> job similarity

In [10]:
combinedjob1=combinedjob.iloc[:20000,:]
combinedjob1['joint'] = combinedjob1["Position"].map(str) + "  " + combinedjob1["Title"] +"  "+ combinedjob1["City"]+"  "+ combinedjob1["Job.Description"]+"  "+ combinedjob1["Education.Required"]
combinedjob1['joint'] = combinedjob1['joint'].str.replace('[^a-zA-Z \n\.]',"")
combinedjob1['joint'] = combinedjob1['joint'].str.lower()
combinedjob1 = combinedjob1[['Job.ID',"Title", 'joint']]
combinedjob1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we c

,Job.ID,Title,joint
0,111,Server @ Tacolicious,server server tacolicious palo alto tacoli...
1,113,Kitchen Staff/Chef @ Claude Lane,kitchen staffchef kitchen staffchef claude l...
2,117,Bartender @ Machka Restaurants Corp.,bartender bartender machka restaurants corp....
3,121,Server @ Teriyaki House,server server teriyaki house brisbane se...
4,127,Kitchen Staff/Chef @ Rosa Mexicano - Sunset,kitchen staffchef kitchen staffchef rosa mex...


In [11]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2), stop_words='english', max_features=10000)
tfidf_matrix = tf.fit_transform(combinedjob1['joint'])


In [12]:
tfidf_matrix.shape

(20000, 10000)

In [13]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [91]:
titles = combinedjob1['Title']
indices = pd.Series(combinedjob1.index, index=combinedjob1['Title'])
len(titles)

20000

In [92]:
def get_recommendations(title):
    if len(str(type(indices[title])))> 21:
      idx = indices[title][0]
    else:
      idx = indices[title]
    #print (idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print (sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    job_indices = [i[0] for i in sim_scores]
    return titles.iloc[job_indices].head(5)

In [93]:
get_recommendations("Server @ Tacolicious")

0                   Server @ Tacolicious
13453                  Server @ Gonpachi
3                Server @ Teriyaki House
10778     Server @ Far Niente Ristorante
10783    Server @ La Fontaine Restaurant
Name: Title, dtype: object

In [94]:
indicess = pd.DataFrame(combinedjob1.index, index=combinedjob1['Title'])

In [95]:
from statistics import mean 
def accuracy(title):
    matches = []
    for job in get_recommendations(title):
        matches.append(len(set(combinedjob1.iloc[indicess.index.get_loc(job)]['joint']) & set(combinedjob1.iloc[indicess.index.get_loc(title)]['joint'])))
    return  mean(matches)/max(matches)

In [96]:
accuracy('Server @ Tacolicious')

0.9583333333333334

In [97]:
def get_user_id(job_list):
    jobs_userwise = jobview['Title'].isin(job_list)
    df1 = pd.DataFrame(data = jobview[jobs_userwise], columns=['Applicant.ID'])
    userlist = df1['Applicant.ID'].tolist()
    userlist = jobview['Applicant.ID'].isin(userlist) 
    df_temp = pd.DataFrame(data = jobview[userlist], columns=['Applicant.ID'])
    return df_temp.drop_duplicates()

In [98]:
get_user_id(get_recommendations("Administrative experience in Life Insurance @ OfficeTeam"))

,Applicant.ID
5503,12733
5891,12984


In [100]:
titles2=combinedjob1['Title'].unique()

In [101]:
df1=pd.DataFrame()
count=0
for i in titles2:
  df=get_user_id(get_recommendations(i))
  df['Tilte']=i
  a=combinedjob["Job.ID"][combinedjob['Title']==i].tolist()[0]
  df['JobID']=a
  df['City']=combinedjob["City"][combinedjob['Job.ID']==a].tolist()[0]
  df['State']=combinedjob["State.Name"][combinedjob['Job.ID']==a].tolist()[0]
  count+=1
  df=pd.DataFrame(df)
  df1=df1.append(df)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [102]:
df1.drop_duplicates().sort_values(by=["Applicant.ID", "JobID"])

,Applicant.ID,Tilte,JobID,City,State
8878,42,1418BR Seasonal Telecollector @ We Energies,147347,Milwaukee,Wisconsin
8878,42,Driving Partner @ Uber,153197,San Francisco,California
8721,153,Staff Accountant @ Accountemps,136420,Fort Myers,Florida
8721,153,Accounts Receivable Clerk @ Accountemps,136442,Hauppauge,New York
8721,153,Accounts Receivable and Billing Specialist @ A...,136446,Woburn,Massachusetts
...,...,...,...,...,...
8711,14546,Personal Driver/Chauffeur @ Quest LLC,227061,los angeles,California
8714,14547,Personal Assistant/ Administrative Assistant @...,231297,New York,New York
8716,14549,Mandarin-English Bilingual Speaking Required -...,146922,Costa Mesa,California
8716,14549,Louis Vuitton Sales Professional Part Time: Bl...,247064,New York,New York


## cosine similarity  ==> user similarity

In [104]:
jobview['pos_com_city'] = jobview["Position"].map(str) + "  " + jobview["Company"] +"  "+ jobview["City"]

jobview['pos_com_city'] = jobview['pos_com_city'].str.replace('[^a-zA-Z \n\.]',"")

jobview['pos_com_city'] = jobview['pos_com_city'].str.lower()

jobview2 = jobview[['Applicant.ID','pos_com_city']]

jobview2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,Applicant.ID,pos_com_city
0,10000,cashiers valets needed wallypark newark
1,10000,macys seasonal retail fragrance cashier gard...
2,10001,part time showroom sales cashier grizzly ind...
3,10002,event specialist part time advantage sales m...
4,10002,bonefish kitchen staff bonefish grill green...


In [105]:
experience = experience[['Applicant.ID','Position.Name']]

#cleaning the text
experience['Position.Name'] = experience['Position.Name'].str.replace('[^a-zA-Z \n\.]',"")
experience['Position.Name'] = experience['Position.Name'].str.lower()
experience = experience.fillna(" ")
experience.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Applicant.ID,Position.Name
0,10001,account manager sales administration quality...
1,10001,electronics technician item master controller
2,10001,machine operator
3,10003,maintenance technician
4,10003,electrical helper


In [106]:
experience = experience.groupby('Applicant.ID', sort=False)['Position.Name'].apply(' '.join).reset_index()
experience.head(20)

,Applicant.ID,Position.Name
0,10001,account manager sales administration quality...
1,10003,maintenance technician electrical helper tech...
2,10004,shift superviveur
3,10005,all star houseman
4,10007,bartender bar manager head bartender bartender
5,10008,nonfood stocker photo registrar registrar cash...
6,1001,sales associate
7,10011,driver helper cashier customer assistant
8,10016,host
9,10020,it technician ii information technology specia...


In [107]:
POI = POI.sort_values(by='Applicant.ID')
POI = POI.drop('Updated.At', 1)
POI = POI.drop('Created.At', 1)

#cleaning the text
POI['Position.Of.Interest']=POI['Position.Of.Interest'].str.replace('[^a-zA-z \n\.]',"")
POI['Position.Of.Interest']=POI['Position.Of.Interest'].str.lower()
POI = POI.fillna(" ")
POI = POI.groupby('Applicant.ID', sort=True)['Position.Of.Interest'].apply(' '.join).reset_index()
POI.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Applicant.ID,Position.Of.Interest
0,96,server
1,153,barista host server sales rep customer service...
2,256,host production area sales rep customer servic...
3,438,customer service rep barista host server
4,568,receptionist customer service rep book keeper


In [108]:
#merging jobs and experience dataframes
out_joint_jobs = jobview2.merge(experience, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
print(out_joint_jobs.shape)
out_joint_jobs = out_joint_jobs.fillna(' ')
out_joint_jobs = out_joint_jobs.sort_values(by='Applicant.ID')
out_joint_jobs.head()

(15383, 3)


,Applicant.ID,pos_com_city,Position.Name
13012,2,,writer for the uloop blog volunteer
13487,3,,prep cook server marketing intern
14628,6,,project assistant
15044,8,,deli clerkserver cashier food prep order taker
12464,11,,cashier


In [109]:
#merging position of interest with existing dataframe
joint_poi_exper_view = out_joint_jobs.merge(POI, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
joint_poi_exper_view = joint_poi_exper_view.fillna(' ')
joint_poi_exper_view = joint_poi_exper_view.sort_values(by='Applicant.ID')
joint_poi_exper_view.head()

,Applicant.ID,pos_com_city,Position.Name,Position.Of.Interest
0,2,,writer for the uloop blog volunteer,
1,3,,prep cook server marketing intern,
2,6,,project assistant,
3,8,,deli clerkserver cashier food prep order taker,
4,11,,cashier,


In [110]:
#combining all the columns

joint_poi_exper_view["pos_com_city1"] = joint_poi_exper_view["pos_com_city"].map(str) + joint_poi_exper_view["Position.Name"] +" "+ joint_poi_exper_view["Position.Of.Interest"]

joint_poi_exper_view.head()

,Applicant.ID,pos_com_city,Position.Name,Position.Of.Interest,pos_com_city1
0,2,,writer for the uloop blog volunteer,,writer for the uloop blog volunteer
1,3,,prep cook server marketing intern,,prep cook server marketing intern
2,6,,project assistant,,project assistant
3,8,,deli clerkserver cashier food prep order taker,,deli clerkserver cashier food prep order taker
4,11,,cashier,,cashier


In [111]:
final_poi_exper_view = joint_poi_exper_view[['Applicant.ID','pos_com_city1']]
final_poi_exper_view.head()

,Applicant.ID,pos_com_city1
0,2,writer for the uloop blog volunteer
1,3,prep cook server marketing intern
2,6,project assistant
3,8,deli clerkserver cashier food prep order taker
4,11,cashier


In [112]:
final_poi_exper_view.columns = ['Applicant_id','pos_com_city1']
final_poi_exper_view = final_poi_exper_view.sort_values(by='Applicant_id')
final_poi_exper_view['pos_com_city1'] = final_poi_exper_view['pos_com_city1'].str.replace('[^a-zA-Z \n\.]',"")
final_poi_exper_view['pos_com_city1'] = final_poi_exper_view['pos_com_city1'].str.lower()
final_poi_exper_view = final_poi_exper_view.groupby('Applicant_id', sort=True)['pos_com_city1'].apply(' '.join).reset_index()
final_poi_exper_view.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,Applicant_id,pos_com_city1
0,2,writer for the uloop blog volunteer
1,3,prep cook server marketing intern
2,6,project assistant
3,8,deli clerkserver cashier food prep order taker
4,11,cashier


In [113]:
user_tfidf = tf.fit_transform(final_poi_exper_view['pos_com_city1'])
cosine_sim2 = linear_kernel(user_tfidf,user_tfidf)

In [114]:
cosine_sim2.shape

(7037, 7037)

In [115]:
final_poi_exper_view = final_poi_exper_view.reset_index()
userid =final_poi_exper_view['Applicant_id']
indices2 = pd.Series(final_poi_exper_view.index, index=final_poi_exper_view['Applicant_id'])
indices2.head()


Applicant_id
2     0
3     1
6     2
8     3
11    4
dtype: int64

In [116]:
def get_recommendations_userwise(userid):
    idx = indices2[userid]
    #print (idx)
    sim_scores2 = list(enumerate(cosine_sim2[idx]))
    #print (sim_scores2)
    sim_scores2 = sorted(sim_scores2, key=lambda x: x[1], reverse=True)
    user_indices = [i[0] for i in sim_scores2]
    return final_poi_exper_view['Applicant_id'].loc[user_indices[0:11]]

In [117]:
print ("-----Top 10 Similar users with Applicant_id: 11------")
get_recommendations_userwise(11)

-----Top 10 Similar users with Applicant_id: 11------


4       11
10      21
14      26
41      73
47      83
59     111
63     118
137    282
270    728
283    774
306    868
Name: Applicant_id, dtype: int64

In [118]:
def get_job_id(usrid_list):
    jobs_userwise2 = jobview['Applicant.ID'].isin(usrid_list) #
    df3 = pd.DataFrame(data = jobview[jobs_userwise2], columns=['Job.ID'])
    joblist = df3['Job.ID'].tolist()
    Job_list = combinedjob['Job.ID'].isin(joblist) 
    df_temp2 = pd.DataFrame(data = combinedjob[Job_list], columns=['Job.ID','Title','Position','City','State.Name'])
    return df_temp2

In [119]:
get_job_id(get_recommendations_userwise(10010))

,Job.ID,Title,Position,City,State.Name
62,134344,Part-time Data Management Associate @ DST,Part-time Data Management Associate,Kansas City,Missouri
2878,141452,Security Officers,Security Officers,Kansas City,Missouri


In [120]:
applicant=final_poi_exper_view['Applicant_id'].unique()
len(titles2)

10788

In [121]:
df5=pd.DataFrame()
count=0
for i in applicant:
  df4=get_job_id(get_recommendations_userwise(i))
  df4['UserID']=i
  df4=pd.DataFrame(df4)
  df5=df5.append(df)

In [122]:
df5

,Applicant.ID,Tilte,JobID,City,State
5503,12733,Administrative experience in Life Insurance @ ...,249527,Miami,Florida
5891,12984,Administrative experience in Life Insurance @ ...,249527,Miami,Florida
5503,12733,Administrative experience in Life Insurance @ ...,249527,Miami,Florida
5891,12984,Administrative experience in Life Insurance @ ...,249527,Miami,Florida
5503,12733,Administrative experience in Life Insurance @ ...,249527,Miami,Florida
...,...,...,...,...,...
5891,12984,Administrative experience in Life Insurance @ ...,249527,Miami,Florida
5503,12733,Administrative experience in Life Insurance @ ...,249527,Miami,Florida
5891,12984,Administrative experience in Life Insurance @ ...,249527,Miami,Florida
5503,12733,Administrative experience in Life Insurance @ ...,249527,Miami,Florida


## user_job similarity

In [ ]:
#!pip install -U turicreate

In [ ]:
import turicreate as tc
jobview_sframe=tc.SFrame.read_csv('/content/drive/My Drive/DATA /Job_Views.csv')
users_sframe=tc.SFrame.read_csv('/content/drive/My Drive/DATA /Experience.csv')
jobs_sframe=tc.SFrame.read_csv('/content/drive/My Drive/DATA /Combined_Jobs_Final.csv')

Finished parsing file /content/drive/My Drive/DATA /Job_Views.csv

Parsing completed. Parsed 100 lines in 0.13942 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,str,str,str,str,str,str,str,int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/drive/My Drive/DATA /Job_Views.csv

Parsing completed. Parsed 12370 lines in 0.08704 secs.

...

Parsing completed. Parsed 100 lines in 1.2602 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,str,str,str,str,str,str,str,float,float,str,str,str,float,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 27898 lines. Lines per second: 18049.7

Finished parsing file /content/drive/My Drive/DATA /Combined_Jobs_Final.csv

Parsing completed. Parsed 84090 lines in 2.59429 secs.

In [ ]:
m = tc.item_similarity_recommender.create(jobview_sframe,
                                    user_id='Applicant.ID',
                                    item_id='Job.ID',
                                    similarity_type="jaccard",
                                    )

Warning: Ignoring columns Title, Position, Company, City, State.Name, State.Code, Industry, View.Start, View.End, View.Duration, Created.At, Updated.At;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 12370 observations with 3448 users and 7047 items.

...

| 529.466ms                           | 100              | 7047            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.533955s

In [ ]:
ID_list=list(pd.unique(jobview_sframe['Applicant.ID']))

In [ ]:
item_sim_recomm = m.recommend(items =ID_list,k=15)

recommendations finished on 1000/3448 queries. users per second: 23365.6

recommendations finished on 2000/3448 queries. users per second: 22050.2

recommendations finished on 3000/3448 queries. users per second: 23072.8

In [ ]:
item_sim_recomm[["Applicant.ID","Job.ID"]]

Applicant.ID,Job.ID
10000,14272
10000,14251
10000,14252
10000,14254
10000,14255
10000,14256
10000,14257
10000,14258
10000,14259
10000,14260


In [ ]:
sf = tc.SFrame('https://static.turi.com/datasets/audioscrobbler')
train, test = tc.recommender.util.random_split_by_user(sf)
b = tc.recommender.create(train)
b.evaluate_precision_recall(test)

Downloading https://static.turi.com/datasets/audioscrobbler/dir_archive.ini to /var/tmp/turicreate-root/8182/6ee8601d-66a2-4a07-821e-f57e86f2de19.ini

Downloading https://static.turi.com/datasets/audioscrobbler/objects.bin to /var/tmp/turicreate-root/8182/d495fe66-7054-4a8b-a6cb-b53208c8d214.bin

Downloading https://static.turi.com/datasets/audioscrobbler/m_74b0dc51.frame_idx to /var/tmp/turicreate-root/8182/8c589749-3ba7-406b-861f-b6be47e27d85.frame_idx

Downloading https://static.turi.com/datasets/audioscrobbler/m_74b0dc51.sidx to /var/tmp/turicreate-root/8182/5fc78860-ede6-403f-a66a-167eb16f3301.sidx

Downloading https://static.turi.com/datasets/audioscrobbler/m_74b0dc51.0000 to /var/tmp/turicreate-root/8182/1845d28a-9618-441c-90ee-8eb0cd2f45db.0000

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.90552s

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 9180
 
 Data:
 +---------+--------+-----------+--------+-------+
 | user_id | cutoff | precision | recall | count |
 +---------+--------+-----------+--------+-------+
 | 2174493 |   1    |    0.0    |  0.0   |   3   |
 | 2174493 |   2    |    0.0    |  0.0   |   3   |
 | 2174493 |   3    |    0.0    |  0.0   |   3   |
 | 2174493 |   4    |    0.0    |  0.0   |   3   |
 | 2174493 |   5    |    0.0    |  0.0   |   3   |
 | 2174493 |   6    |    0.0    |  0.0   |   3   |
 | 2174493 |   7    |    0.0    |  0.0   |   3   |
 | 2174493 |   8    |    0.0    |  0.0   |   3   |
 | 2174493 |   9    |    0.0    |  0.0   |   3   |
 | 2174493 |   10   |    0.0    |  0.0   |   3   |
 +---------+--------+-----------+--------+-------+
 [9180 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns

In [ ]:
sf

user_id,item_id,target
1001606,Raver's Nature,2
1002598,Raver's Nature,1
1009768,Raver's Nature,29
1020855,Raver's Nature,9
1024677,Raver's Nature,1
1025593,Raver's Nature,3
1036036,Raver's Nature,51
1037250,Raver's Nature,3
1037372,Raver's Nature,1
1037666,Raver's Nature,1


In [ ]:
n= tc.item_similarity_recommender.create(jobview_sframe,
                                    user_id='Applicant.ID',
                                    item_id='Job.ID',
                                    similarity_type="jaccard",
                                    )
nn = n.get_similar_items()

Warning: Ignoring columns Title, Position, Company, City, State.Name, State.Code, Industry, View.Start, View.End, View.Duration, Created.At, Updated.At;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 12370 observations with 3448 users and 7047 items.

...

| 549.965ms                           | 100              | 7047            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.564728s